In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import json
from datetime import datetime


### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>



In [76]:


def add_data(date_, name_, datas):
    dict_datas = {"января":"01","февраля":"02","марта":"03","апреля":"04","мая":"05","июня":"06",
                  "июля":"07","августа":"08","сентября":"09","октября":"10","ноября":"11","декабря":"12"}
    if "сегодня" in date_:
        date = datetime.now().date()
    elif "вчера" in date_:
        date_ = datetime.now().date() - pd.offsets.Day(1)
    else:
        month = dict_datas[(re.findall(r'\d{1,2}\s(.+)\s\d{4}.+',date_)[0])]
        date = datetime.strptime(re.sub(r'(\d{1,2})\s(.*)\s(\d{4})(.*)', r'\3-'+month+r'-\1',date_), "%Y-%m-%d")
        
    row = {'date': date, 'title': name_.text, 'link': name_.get('href')}
    datas = pd.concat([datas, pd.DataFrame([row])]) 
    return datas

def pars_habr(KEYWORDS):  
    datas = pd.DataFrame()
    res = requests.get('https://habr.com/ru/all/')
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_time = post.find('span', class_='post__time')
        post_link = post.find('a', class_='post__title_link')
        post_text = ((post.find('div', class_='post__text')).text).lower()
        post_hubs = post.find_all('a', class_='hub-link')
        
        if any([KEYWORD in post_text for KEYWORD in KEYWORDS]) or any([KEYWORD in ((post_link.text).lower()) for KEYWORD in KEYWORDS]):
            datas = add_data(post_time.text,post_link,datas)
        else:
            for hub in post_hubs:
                if any([KEYWORD in ((hub.text).lower()) for KEYWORD in KEYWORDS]):
                    datas = add_data(post_time.text, post_link, datas)
    return(datas)

def main():
    KEYWORDS = ['python', 'парсинг','agile', 'тестирование','здоровье']
    data_frame = pars_habr(KEYWORDS)
    print(data_frame)
main()

         date                                              title  \
0  2021-07-05  Спектрометр своими руками за 5 долларов и немн...   
0  2021-07-05  Нейродайджест: главное из области машинного об...   
0  2021-07-05  Учимся читать код, изучая стандартную библиоте...   

                                                link  
0  https://habr.com/ru/company/skillfactory/blog/...  
0                   https://habr.com/ru/post/566236/  
0    https://habr.com/ru/company/vdsina/blog/566134/  


#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>



In [75]:
def add_data_with_text(date_, name_, datas, text):
    dict_datas = {"января":"01","февраля":"02","марта":"03","апреля":"04","мая":"05","июня":"06",
                  "июля":"07","августа":"08","сентября":"09","октября":"10","ноября":"11","декабря":"12"}
    if "сегодня" in date_:
        date = datetime.now().date()
    elif "вчера" in date_:
        date_ = datetime.now().date() - pd.offsets.Day(1)
    else:
        month = dict_datas[(re.findall(r'\d{1,2}\s(.+)\s\d{4}.+',date_)[0])]
        date = datetime.strptime(re.sub(r'(\d{1,2})\s(.*)\s(\d{4})(.*)', r'\3-'+month+r'-\1',date_), "%Y-%m-%d")
    row = {'date': date, 'title': name_.text, 'link': name_.get('href'), 'text': text}
    datas = pd.concat([datas, pd.DataFrame([row])]) 
    return datas


def get_post_text( link):
    soup = BeautifulSoup((requests.get(link)).text, 'html.parser')
    post = soup.find('div', class_='post__body')
    return(post.text.lower())


def pars_habr_posts(KEYWORDS):  
    datas = pd.DataFrame()
    res = requests.get('https://habr.com/ru/all/')
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_='post')
    for post in posts:
        post_time = post.find('span', class_='post__time')
        post_link = post.find('a', class_='post__title_link')
        post_text = get_post_text( post_link.get('href'))
        post_hubs = post.find_all('a', class_='hub-link')
        if any([KEYWORD in post_text for KEYWORD in KEYWORDS]) or any([KEYWORD in ((post_link.text).lower()) for KEYWORD in KEYWORDS]) :
            datas = add_data_with_text(post_time.text,post_link,datas,post_text) 
        else:
            for hub in post_hubs:
                if any([KEYWORD in ((hub.text).lower()) for KEYWORD in KEYWORDS]):
                    datas = add_data_with_text(post_time.text, post_link, datas,post_text)
    return(datas)

def main():
    KEYWORDS = ['python', 'парсинг','agile', 'тестирование','здоровье']
    data_frame = pars_habr_posts(KEYWORDS)
    print(data_frame)
main()


         date                                              title  \
0  2021-07-05  Спектрометр своими руками за 5 долларов и немн...   
0  2021-07-05  Нейродайджест: главное из области машинного об...   
0  2021-07-05  Учимся читать код, изучая стандартную библиоте...   
0  2021-07-05              Издательство Питер. Колонка редактора   
0  2021-07-05  Уроки, которые мы вынесли из опыта управления ...   

                                                link  \
0  https://habr.com/ru/company/skillfactory/blog/...   
0                   https://habr.com/ru/post/566236/   
0    https://habr.com/ru/company/vdsina/blog/566134/   
0     https://habr.com/ru/company/piter/blog/566196/   
0   https://habr.com/ru/company/timeweb/blog/566212/   

                                                text  
0  \nв освоении физики лабораторные эксперименты ...  
0  \nначнем подборку с новостей из области nlp. я...  
0  \n\n\r\nитак, вы уже продвинутый новичок — вы ...  
0  \n\r\nпривет, хаброжители! пред

### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.



In [56]:

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
EMAIL = ['n.bor98@yandex.ru', 'xxx@x.ru', 'yyy@y.com']
params = {
    'emailAddresses': EMAIL
}
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
}
req = requests.post(URL, data=json.dumps(params), headers=headers)
datas = pd.DataFrame()  
diction  = json.loads(req.text)
for mail in EMAIL:
    if len(diction['summary'][mail]['breaches'])>0:
        for breach in diction['summary'][mail]['breaches']:
            row = {'mail': mail, 'date': diction['breaches'][str(breach)]['publishDate'],'site': diction['breaches'][str(breach)]['site'], 'description': diction['breaches'][str(breach)]['description']}
            datas = pd.concat([datas, pd.DataFrame([row])]) 
print(datas)

        mail                  date                  site  \
0   xxx@x.ru  2016-10-21T00:00:00Z             adobe.com   
0   xxx@x.ru  2016-10-29T00:00:00Z                vk.com   
0   xxx@x.ru  2016-10-23T00:00:00Z             imesh.com   
0   xxx@x.ru  2017-01-31T00:00:00Z      cdprojektred.com   
0   xxx@x.ru  2017-02-14T00:00:00Z         cfire.mail.ru   
0   xxx@x.ru  2017-02-14T00:00:00Z        parapa.mail.ru   
0  yyy@y.com  2016-10-21T00:00:00Z          linkedin.com   
0  yyy@y.com  2016-10-21T00:00:00Z             adobe.com   
0  yyy@y.com  2016-10-23T00:00:00Z             imesh.com   
0  yyy@y.com  2016-10-24T00:00:00Z           dropbox.com   
0  yyy@y.com  2017-03-15T00:00:00Z        globalreach.eu   
0  yyy@y.com  2017-03-01T00:00:00Z          rayli.com.cn   
0  yyy@y.com  2017-03-24T00:00:00Z             youku.com   
0  yyy@y.com  2017-11-04T00:00:00Z        myheritage.com   
0  yyy@y.com  2018-02-18T00:00:00Z            netlog.com   
0  yyy@y.com  2019-06-13T00:00:00Z      

#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>